In [1]:
# run initial imports
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go

from scipy.integrate import odeint
from scipy.optimize import curve_fit

from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets

import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

import os
from datetime import datetime

In [2]:
from folium.plugins.marker_cluster import MarkerCluster
from folium.utilities import if_pandas_df_convert_to_numpy, validate_location

from jinja2 import Template


class CustomFastMarkerCluster(MarkerCluster):
    """
    Add marker clusters to a map using in-browser rendering.
    Using FastMarkerCluster it is possible to render 000's of
    points far quicker than the MarkerCluster class.
    Be aware that the FastMarkerCluster class passes an empty
    list to the parent class' __init__ method during initialisation.
    This means that the add_child method is never called, and
    no reference to any marker data are retained. Methods such
    as get_bounds() are therefore not available when using it.
    Parameters
    ----------
    data: list of list with values
        List of list of shape [[lat, lon], [lat, lon], etc.]
        When you use a custom callback you could add more values after the
        lat and lon. E.g. [[lat, lon, 'red'], [lat, lon, 'blue']]
    callback: string, optional
        A string representation of a valid Javascript function
        that will be passed each row in data. See the
        FasterMarkerCluster for an example of a custom callback.
    name : string, optional
        The name of the Layer, as it will appear in LayerControls.
    overlay : bool, default True
        Adds the layer as an optional overlay (True) or the base layer (False).
    control : bool, default True
        Whether the Layer will be included in LayerControls.
    show: bool, default True
        Whether the layer will be shown on opening (only for overlays).
    icon_create_function : string, default None
        Override the default behaviour, making possible to customize
        markers colors and sizes.
    **kwargs
        Additional arguments are passed to Leaflet.markercluster options. See
        https://github.com/Leaflet/Leaflet.markercluster
    """
    _template = Template(u"""
        {% macro script(this, kwargs) %}
            var {{ this.get_name() }} = (function(){
                {{ this.callback }}
                var data = {{ this.data|tojson }};
                var cluster = L.markerClusterGroup({{ this.options|tojson }});
                {%- if this.icon_create_function is not none %}
                cluster.options.iconCreateFunction =
                    {{ this.icon_create_function.strip() }};
                {%- endif %}
                for (var i = 0; i < data.length; i++) {
                    var row = data[i];
                    var marker = callback(row);
                    marker.addTo(cluster);
                }
                cluster.addTo({{ this._parent.get_name() }});
                return cluster;
            })();
        {% endmacro %}""")

    def __init__(self, data, callback=None, options=None,
                 name=None, overlay=True, control=True, show=True, icon_create_function=None, **kwargs):
        if options is not None:
            kwargs.update(options)  # options argument is legacy
        super(CustomFastMarkerCluster, self).__init__(name=name, overlay=overlay,
                                                control=control, show=show,
                                                icon_create_function=icon_create_function,
                                                **kwargs)
        self._name = 'CustomFastMarkerCluster'
        data = if_pandas_df_convert_to_numpy(data)
        self.data = [[*validate_location(row[:2]), *row[2:]]  # noqa: E999
                     for row in data]

        if callback is None:
            self.callback = """
                var callback = function (row) {
                    var marker = L.marker(new L.LatLng(row[0], row[1]));
                    return marker;
                };"""
        else:
            self.callback = 'var callback = {};'.format(callback)

In [3]:
NOW = datetime.now()
input_dir = './data/'
combined = pd.read_csv(input_dir + 'web_cases.csv')
combined['Last Update'] = pd.to_datetime(combined['Last_Update'])
combined['Latitude'] = combined['Lat']
combined['Longitude'] = combined['Long_']
combined['Province/State'] = combined['Province_State']
combined['Country/Region'] = combined['Country_Region']
combined = combined.sort_values('Last Update').reset_index(drop=True)

#extract lat, lon for non mapped stuff
locations = combined.groupby(['Country/Region', 'Province/State'])['Latitude', 'Longitude'].mean().reset_index()
locations.columns = ['Country/Region', 'Province/State', 'Latitude_Lookup', 'Longitude_Lookup']

combined = pd.merge(left=combined, right=locations, left_on=['Country/Region', 'Province/State'], right_on=['Country/Region', 'Province/State'], how='left')
combined['Latitude'] = combined['Latitude'].fillna(combined['Latitude_Lookup'])
combined['Longitude'] = combined['Longitude'].fillna(combined['Longitude_Lookup'])
combined['Province/State'] = combined['Province/State'].fillna(combined['Country/Region'])
del combined['Latitude_Lookup'] 
del combined['Longitude_Lookup'] 
combined = combined.fillna(0)

In [4]:
#do a little re-ordering
combined = combined[['Last Update', 'Latitude', 'Longitude', 'Country/Region', 'Province/State','Active', 'Confirmed','Recovered','Deaths']].copy().reset_index(drop=True)
#combined = combined[combined['Country/Region'] != 0]
#combined

In [5]:
latest = combined.sort_values('Last Update').groupby(['Latitude','Longitude']).tail(1).copy().reset_index(drop=True)
latest['Active'] = latest['Confirmed'] - latest['Recovered'] - latest['Deaths']
#latest

In [6]:
deaths = latest[latest['Deaths']>0]
deaths = pd.DataFrame(deaths.values.repeat(deaths.Deaths, axis=0), columns=deaths.columns)

confirmed = latest[latest['Confirmed']>0]
confirmed = pd.DataFrame(confirmed.values.repeat(confirmed.Confirmed, axis=0), columns=confirmed.columns)

us_confirmed = latest[latest['Confirmed']>0]
us_confirmed = us_confirmed[us_confirmed['Country/Region'] == 'US']
us_confirmed = pd.DataFrame(us_confirmed.values.repeat(us_confirmed.Confirmed, axis=0), columns=us_confirmed.columns)

recovered = latest[latest['Recovered']>0]
recovered = pd.DataFrame(recovered.values.repeat(recovered.Recovered, axis=0), columns=recovered.columns)

active = latest[latest['Active']>0]
active = pd.DataFrame(active.values.repeat(active.Active, axis=0), columns=active.columns)

us_active = latest[latest['Active']>0]
us_active = us_active[us_active['Country/Region'] == 'US']
us_active = pd.DataFrame(us_active.values.repeat(us_active.Active, axis=0), columns=us_active.columns)

na = latest[latest['Active'] > 0]
na_active = na[na['Country/Region'] == 'US']
na_active = na_active.append(na[na['Country/Region'] == 'Mexico'])
na_active = na_active.append(na[na['Country/Region'] == 'Canada'])
na_active = pd.DataFrame(na_active.values.repeat(na_active.Active, axis=0), columns=na_active.columns)

In [ ]:
folium_map = folium.Map(location=[38.826555, -100.244867],
                        zoom_start=2,
                        tiles=None)
folium.TileLayer('CartoDB dark_matter', name='Base Layer').add_to(folium_map)
folium.TileLayer('openstreetmap', name='Openstreetmap').add_to(folium_map)
folium.TileLayer('Stamen Terrain', name='Terrain').add_to(folium_map)

FastMarkerCluster(data=list(zip(active['Latitude'].values, active['Longitude'].values)), name='Active Cases').add_to(folium_map)

lc = folium.LayerControl(collapsed=False).add_to(folium_map)
folium_map